# **Attendance Data Consolidation and Transformation with Pandas**

**Note:** This notebook contains a modified version of codes I developed to improve an existing process during my previous employment. The techniques and data formats applied here are similar to those used in the real-world scenario, but the actual data, including names, numbers, and values, have been randomly generated for the purpose of illustration and confidentiality. The modifications aim to showcase the approach and techniques applied in the original context.

### **Background**

Code in this notebook was used to streamline the attendance reporting process that I used to perform on a daily basis. The daily task involved transforming schedule and phone data of over 500 service desk employees into a format that is uploaded to SQL for use in an attendance dashboard.

#### **Goal**

The goal is to generate two outputs from raw data in a quick and efficient manner.
1. Consolidated schedule data - Contains:
  * Date
  * Start time
  * End time  
  * Employee number
  * Code: 1 (Off), 2 (Shift), 3 (Training), 4 (Vacation), 5 (PTO)

2. Employee login logout data - Contains:
  * Employee name
  * Date
  * Login time
  * Logout time
  * Employee number

#### **Raw Data**

##### **Schedule data**

Schedule data is provided separately by each department. They all use the same Excel file template.

![picture](https://drive.google.com/uc?export=view&id=1OsR9ezvDlW5Pw3ORYCtt4g9LNmYaK_TD)

##### **Phone data**

Employee login/logout data is obtained from the software phone system in Excel format. A day may have multiple login and logout times.

![picture](https://drive.google.com/uc?export=view&id=15MZvNlQopQJdMIvQ3qmwzsnTW-qINl8u)

##### **Employee List**

The employee list is also saved as an Excel file. It contains employee IDs, Z IDs and phone IDs.

![picture](https://drive.google.com/uc?export=view&id=1_as6A6QaJtbvSfLDHSRXP7j57ovyaJ-E)



### **Part 1: Schedule Extraction**

First, we import the libraries. Aside from pandas, we use datetime for a few datetime operations, and os to read multiple Excel files from a folder.

We then assign a date to a variable. We set the data type to datetime because pandas' read_excel converts date values to this format.

In [1]:
import pandas as pd
import datetime as dt
import os

year = 2023
month = 12
date = 6

selected_date = dt.datetime(year, month, date) # Assign date to variable
print(selected_date)

2023-12-06 00:00:00


We then determine the folder location and display the list of Excel files in the folder.

In [2]:
schedule_path = '/content/drive/MyDrive/attendance/schedule' # Location of schedule files

schedule_files = [file for file in os.listdir(schedule_path) if file.endswith('.xlsx')] # Read folder for list of xlsx files

for file in schedule_files:
    print(file) # Show list of xlsx files

Account3.xlsx
Account1.xlsx
Account2.xlsx


Based on the list of Excel files, we assign the list of accounts that we will be working on into a list.

In [3]:
# List of accounts for schedule extraction
accounts = ['Account1', 'Account2', 'Account3']

We then combine data from the corresponding Excel files into a dataframe.

In [4]:
combined_df = []

for account in accounts:
    file_df = pd.read_excel(os.path.join(schedule_path, f'{account}.xlsx'), header=0)
    combined_df.append(file_df)

df = pd.concat(combined_df, ignore_index=True) # Combine data from every file into 1 df

df.head()

,Account,Site,Employee First Name,Employee Last Name,Employee Full Name,Z ID,ROLE,Time In,Time Out,WFH or Onsite,2023-12-04 00:00:00,2023-12-05 00:00:00,2023-12-06 00:00:00,2023-12-07 00:00:00,2023-12-08 00:00:00,2023-12-09 00:00:00,2023-12-10 00:00:00
0,Account1,Site B,Leo,Anderson,"Anderson, Leo",Z920211,L1,21:30:00,06:30:00,Onsite,21:30:00,21:30:00,VL,VL,VL,OFF,OFF
1,Account1,Site B,Sophia,Archer,"Archer, Sophia",Z431118,L1,21:30:00,06:30:00,Onsite,21:30:00,21:30:00,21:30:00,21:30:00,21:30:00,OFF,OFF
2,Account1,Site B,Emma,Barrett,"Barrett, Emma",Z895504,L1,22:30:00,07:30:00,Onsite,22:30:00,22:30:00,22:30:00,22:30:00,VL,OFF,OFF
3,Account1,Site A,Nathan,Barrett,"Barrett, Nathan",Z450462,L1,18:00:00,03:00:00,WFH,18:00:00,18:00:00,VL,VL,VL,OFF,OFF
4,Account1,Site B,Samuel,Bennett,"Bennett, Samuel",Z881393,L1,16:00:00,01:00:00,Onsite,09:00:00,09:30:00,16:00:00,16:00:00,16:00:00,OFF,OFF


We then remove whitespaces from all Z IDs as unnecessary whitespaces could cause issues with some operations later. We then filter out based on the value under "Role", as we are dealing with only service desk agents. We also filter out entries that are blank under our selected date.

In [5]:
df['Z ID'] = df['Z ID'].str.strip() # Remove whitespaces from all Z IDs

# Take only rows that are L1, L1.5 or L2 under 'ROLE' and not NA or blank under the selected date. Select only relevant columns
df = df[df['ROLE'].isin(['L1', 'L1.5', 'L2']) & df[selected_date].notna()][['Account', 'Employee Full Name', 'Z ID', 'ROLE', 'Time In', selected_date]]

df.head()

,Account,Employee Full Name,Z ID,ROLE,Time In,2023-12-06 00:00:00
0,Account1,"Anderson, Leo",Z920211,L1,21:30:00,VL
1,Account1,"Archer, Sophia",Z431118,L1,21:30:00,21:30:00
2,Account1,"Barrett, Emma",Z895504,L1,22:30:00,22:30:00
3,Account1,"Barrett, Nathan",Z450462,L1,18:00:00,VL
4,Account1,"Bennett, Samuel",Z881393,L1,16:00:00,16:00:00


We then check Z IDs for duplicates. Sometimes, an employee gets transferred to a different account but the schedule file does not get updated on time.

In [6]:
df['Z ID'].value_counts() # Check for duplicate Z IDs

Z768155    2
Z284698    1
Z399840    1
Z735944    1
Z739094    1
          ..
Z855300    1
Z453011    1
Z668825    1
Z217526    1
Z735420    1
Name: Z ID, Length: 84, dtype: int64

In this case, we check the details of Z768155 and determine which entry is incorrect.

In [7]:
df[df['Z ID'] == 'Z768155']

,Account,Employee Full Name,Z ID,ROLE,Time In,2023-12-06 00:00:00
42,Account1,"Sullivan, Zoey",Z768155,L2,21:30:00,21:30:00
76,Account2,"Sullivan, Zoey",Z768155,L1,None,OFF


The duplicate Z ID with the wrong schedule is dropped.

In [8]:
df = df.drop(76)
df[df['Z ID'] == 'Z768155']

,Account,Employee Full Name,Z ID,ROLE,Time In,2023-12-06 00:00:00
42,Account1,"Sullivan, Zoey",Z768155,L2,21:30:00,21:30:00


The schedule file only contains employee Z IDs. We will need both their employee IDs and phone IDs as well for the later operations. We make a dataframe of the contents of the emplist file. We drop columns that we do not need.

In [9]:
# Get employee numbers from emplist file

emp_path = '/content/drive/MyDrive/attendance/EmpList.xlsx' # Location of emplist file

emp_df = pd.read_excel(emp_path)
emp_df.drop(columns=['Name', 'Account'], inplace=True)
emp_df.head()

,Employee ID,Z ID,Phone ID
0,1089037,Z920211,39105058
1,1172007,Z431118,25393153
2,1323496,Z895504,34591484
3,1407756,Z450462,20152612
4,1544048,Z881393,39176237


We merge the emplist dataframe with the original dataframe with a left join. Now, df has employee IDs and phone IDs.

In [10]:
df = df.merge(emp_df, how = 'left', on = 'Z ID')

df.head()

,Account,Employee Full Name,Z ID,ROLE,Time In,2023-12-06 00:00:00,Employee ID,Phone ID
0,Account1,"Anderson, Leo",Z920211,L1,21:30:00,VL,1089037,39105058
1,Account1,"Archer, Sophia",Z431118,L1,21:30:00,21:30:00,1172007,25393153
2,Account1,"Barrett, Emma",Z895504,L1,22:30:00,22:30:00,1323496,34591484
3,Account1,"Barrett, Nathan",Z450462,L1,18:00:00,VL,1407756,20152612
4,Account1,"Bennett, Samuel",Z881393,L1,16:00:00,16:00:00,1544048,39176237


We review the values of all entries under the column of our selected date to see if there's any invalid entries. We expect mostly time values, or the strings OFF, VL, or PTO.

In [11]:
df[selected_date].value_counts() # Check values from selected date

21:30:00    13
OFF          8
20:00:00     8
22:00:00     7
21:00:00     6
19:00:00     5
14:00:00     4
VL           4
23:00:00     4
06:00:00     3
15:00:00     3
05:00:00     3
16:00:00     2
17:00:00     2
07:00:00     2
22:30:00     2
18:00:00     2
00:00:00     1
09:00:00     1
13:00:00     1
VK           1
11:00:00     1
03:00:00     1
Name: 2023-12-06 00:00:00, dtype: int64

We see a value that is not one of our expected values. We review the row.

In [12]:
df[df[selected_date] == 'VK']

,Account,Employee Full Name,Z ID,ROLE,Time In,2023-12-06 00:00:00,Employee ID,Phone ID
69,Account2,"Donovan, Oliver",Z201091,L2,05:00:00,VK,8682367,34675814


We manually correct the value of this row.

In [13]:
df.loc[69, selected_date] = 'VL'
df.loc[69]

Account                       Account2
Employee Full Name     Donovan, Oliver
Z ID                           Z201091
ROLE                                L2
Time In                       05:00:00
2023-12-06 00:00:00                 VL
Employee ID                    8682367
Phone ID                      34675814
Name: 69, dtype: object

We add functions to determine the values we need for our final table for this part. For start time, we either take the time value under the date. If it's not a time value, we take the value under "Time In" and if that's also not a time value, we set it to zero. We then set end time as start time + 9 hours. For the code, we use an if else statement based on the value under our selected date.

In [14]:
# Function for schedule start time - Use either start time for the selected date, Time In, or 0:00

def start_time(row):
    if isinstance(row[selected_date], dt.time):
        return row[selected_date]
    elif isinstance(row['Time In'], dt.time):
        return row['Time In']
    else:
        return dt.time(0, 0, 0)

# Function for end time - Add 9 hours to start time

def end_time(time):
    new_time = dt.datetime(1, 1, 1, time.hour, time.minute, time.second) + dt.timedelta(hours=9)
    return new_time.time()

# Function for sql plan

def code_value(value):
    if isinstance(value, str):
        if value == 'PTO':
            return 5
        elif value == 'VL':
            return 4
        else:
            return 1
    else:
        return 2


df['start_time'] = df.apply(start_time, axis=1)
df['end_time'] = df['start_time'].apply(lambda x: end_time(x))
df['code'] = df[selected_date].apply(code_value)
df['date'] = selected_date.strftime("%m/%d/%Y")

df.head()

,Account,Employee Full Name,Z ID,ROLE,Time In,2023-12-06 00:00:00,Employee ID,Phone ID,start_time,end_time,code,date
0,Account1,"Anderson, Leo",Z920211,L1,21:30:00,VL,1089037,39105058,21:30:00,06:30:00,4,12/06/2023
1,Account1,"Archer, Sophia",Z431118,L1,21:30:00,21:30:00,1172007,25393153,21:30:00,06:30:00,2,12/06/2023
2,Account1,"Barrett, Emma",Z895504,L1,22:30:00,22:30:00,1323496,34591484,22:30:00,07:30:00,2,12/06/2023
3,Account1,"Barrett, Nathan",Z450462,L1,18:00:00,VL,1407756,20152612,18:00:00,03:00:00,4,12/06/2023
4,Account1,"Bennett, Samuel",Z881393,L1,16:00:00,16:00:00,1544048,39176237,16:00:00,01:00:00,2,12/06/2023


We then create a new dataframe keeping only the columns we need.

In [15]:
schedule_data = df[['date', 'start_time', 'end_time', 'Employee ID', 'code']]
schedule_data.head(10)

,date,start_time,end_time,Employee ID,code
0,12/06/2023,21:30:00,06:30:00,1089037,4
1,12/06/2023,21:30:00,06:30:00,1172007,2
2,12/06/2023,22:30:00,07:30:00,1323496,2
3,12/06/2023,18:00:00,03:00:00,1407756,4
4,12/06/2023,16:00:00,01:00:00,1544048,2
5,12/06/2023,23:00:00,08:00:00,1640652,2
6,12/06/2023,21:30:00,06:30:00,1785955,2
7,12/06/2023,07:00:00,16:00:00,1893881,1
8,12/06/2023,20:00:00,05:00:00,1897692,2
9,12/06/2023,21:30:00,06:30:00,1045515,2


The new dataframe is saved to a csv file.

In [16]:
schedule_data.to_csv('/content/drive/MyDrive/attendance/schedule_data.csv', index=False)

The data is saved. This is the format that our SQL schedule table accepts.

![picture](https://drive.google.com/uc?export=view&id=1oM8zS61S_jrfTV00IVLsadnj0aT7VyCo)

### **Part 2: Login/Logout Data**

First we take create a df with the phone data. To ensure that all possible logins within our 24 hour period is covered, we use phone login/logout data that cover at least 3 days.

In [17]:
phone_data_path = '/content/drive/MyDrive/attendance/PhoneData.xlsx'

phone_data_df = pd.read_excel(phone_data_path) # Create df using phone data

phone_data_df.head()

,Team Name (ID),Agent Name (ID),Agent Session,Login Date,Logout Date,Duration
0,Account1 (7640555),NaN,NaN,NaT,NaT,NaN
1,NaN,"Anderson, Leo (39105058)",NaN,NaT,NaT,NaN
2,NaN,NaN,474816803553,2023-12-04 21:27:48.983,2023-12-05 02:14:17.983,4:46:29
3,NaN,NaN,474816985246,2023-12-05 02:40:30.030,2023-12-05 04:15:05.030,1:34:35
4,NaN,NaN,474817007896,2023-12-05 04:18:04.483,2023-12-05 06:30:15.483,2:12:11


Because of the way this data is formatted, we have to clean it first and ensure that it is properly formatted for our operations later. We do the following:
* Drop unneeded columns
* Fill NAs under Team Name and Agent Name with the nonNA value immediately above them.
* Drop rows with NA under Login Date.
* Extract the number beside the agent names and create a separate column for them. These are the phone IDs.

In [18]:
phone_data_df.drop(columns=['Agent Session', 'Duration'], inplace=True) # Drop Duration Column
phone_data_df['Team Name (ID)'].fillna(method='ffill', inplace=True) # Fill team name NaNs with value immediately above
phone_data_df['Agent Name (ID)'].fillna(method='ffill', inplace=True) # Fill agent name NaNs with value immediately above
phone_data_df.dropna(subset=['Login Date'], inplace=True) # Drop NaTs under Login Date
phone_data_df.reset_index(drop=True, inplace=True)
phone_data_df['Phone ID'] = phone_data_df['Agent Name (ID)'].str.extract(r'\((\d+)\)').astype(int) # Extract phone ID and add column
phone_data_df.head()

,Team Name (ID),Agent Name (ID),Login Date,Logout Date,Phone ID
0,Account1 (7640555),"Anderson, Leo (39105058)",2023-12-04 21:27:48.983,2023-12-05 02:14:17.983,39105058
1,Account1 (7640555),"Anderson, Leo (39105058)",2023-12-05 02:40:30.030,2023-12-05 04:15:05.030,39105058
2,Account1 (7640555),"Anderson, Leo (39105058)",2023-12-05 04:18:04.483,2023-12-05 06:30:15.483,39105058
3,Account1 (7640555),"Anderson, Leo (39105058)",2023-12-05 21:25:26.173,2023-12-06 02:09:02.173,39105058
4,Account1 (7640555),"Anderson, Leo (39105058)",2023-12-06 02:36:29.963,2023-12-06 06:30:19.963,39105058


We return to our original dataframe df and create 2 new columns. These contain our start_time and end_time columns but in pd.Timestamp format. We will use them for timestamp calculations later.

In [19]:
def combine_date_time(row, time_col):
    time_value = row[time_col]
    combined_value = dt.datetime.combine(selected_date.date(), time_value)
    return pd.Timestamp(combined_value)

df['start_timestamp'] = df.apply(combine_date_time, time_col='start_time', axis=1) # Add start time column
df['end_timestamp'] = df['start_timestamp'] + pd.Timedelta(hours=9) # Add end time column based on start time

df.head()

,Account,Employee Full Name,Z ID,ROLE,Time In,2023-12-06 00:00:00,Employee ID,Phone ID,start_time,end_time,code,date,start_timestamp,end_timestamp
0,Account1,"Anderson, Leo",Z920211,L1,21:30:00,VL,1089037,39105058,21:30:00,06:30:00,4,12/06/2023,2023-12-06 21:30:00,2023-12-07 06:30:00
1,Account1,"Archer, Sophia",Z431118,L1,21:30:00,21:30:00,1172007,25393153,21:30:00,06:30:00,2,12/06/2023,2023-12-06 21:30:00,2023-12-07 06:30:00
2,Account1,"Barrett, Emma",Z895504,L1,22:30:00,22:30:00,1323496,34591484,22:30:00,07:30:00,2,12/06/2023,2023-12-06 22:30:00,2023-12-07 07:30:00
3,Account1,"Barrett, Nathan",Z450462,L1,18:00:00,VL,1407756,20152612,18:00:00,03:00:00,4,12/06/2023,2023-12-06 18:00:00,2023-12-07 03:00:00
4,Account1,"Bennett, Samuel",Z881393,L1,16:00:00,16:00:00,1544048,39176237,16:00:00,01:00:00,2,12/06/2023,2023-12-06 16:00:00,2023-12-07 01:00:00


We create 2 new columns: Actual Login and Actual Logout.
Login time is determined by selecting the earliest login timestamp from all instances within an employee's schedule, including up to 4 hours before the scheduled start time. We chose 4 hours to account for possible unplanned changes in employee schedule prompted by business need.
Logout time calculation is similar to Actual Login except we take the latest logout timestamp instead, up to 4 hours after schedule.

In [20]:
# Function to determine earliest login time within period between 4 hours before shift and schedule

def earliest_login(agent_no, login_time, logout_time):
    filtered = phone_data_df[
        (phone_data_df['Phone ID'] == agent_no) &
        (phone_data_df['Login Date'] >= login_time - pd.Timedelta(hours=4)) &
        (phone_data_df['Login Date'] < logout_time)
    ]
    return filtered['Login Date'].min()

df['Actual Login'] = df.apply(lambda row: earliest_login(row['Phone ID'], row['start_timestamp'], row['end_timestamp']), axis=1)

# Function to determine latest logout time within period between schedule to 4 hours after end of shift

def latest_logout(agent_no, login_time, logout_time):
    filtered = phone_data_df[
        (phone_data_df['Phone ID'] == agent_no) &
        (phone_data_df['Logout Date'] <= logout_time + pd.Timedelta(hours=4)) &
        (phone_data_df['Logout Date'] > login_time)
    ]
    return filtered['Logout Date'].max()

df['Actual Logout'] = df.apply(lambda row: latest_logout(row['Phone ID'], row['start_timestamp'], row['end_timestamp']), axis=1)

df.head()

,Account,Employee Full Name,Z ID,ROLE,Time In,2023-12-06 00:00:00,Employee ID,Phone ID,start_time,end_time,code,date,start_timestamp,end_timestamp,Actual Login,Actual Logout
0,Account1,"Anderson, Leo",Z920211,L1,21:30:00,VL,1089037,39105058,21:30:00,06:30:00,4,12/06/2023,2023-12-06 21:30:00,2023-12-07 06:30:00,NaT,NaT
1,Account1,"Archer, Sophia",Z431118,L1,21:30:00,21:30:00,1172007,25393153,21:30:00,06:30:00,2,12/06/2023,2023-12-06 21:30:00,2023-12-07 06:30:00,NaT,NaT
2,Account1,"Barrett, Emma",Z895504,L1,22:30:00,22:30:00,1323496,34591484,22:30:00,07:30:00,2,12/06/2023,2023-12-06 22:30:00,2023-12-07 07:30:00,NaT,NaT
3,Account1,"Barrett, Nathan",Z450462,L1,18:00:00,VL,1407756,20152612,18:00:00,03:00:00,4,12/06/2023,2023-12-06 18:00:00,2023-12-07 03:00:00,NaT,NaT
4,Account1,"Bennett, Samuel",Z881393,L1,16:00:00,16:00:00,1544048,39176237,16:00:00,01:00:00,2,12/06/2023,2023-12-06 16:00:00,2023-12-07 01:00:00,2023-12-06 15:58:33.743,2023-12-07 01:01:43.743


We then create a new dataframe, taking the columns we need from df. We change the format of Actual Login and Actual Logout to just HH:MM.

In [21]:
lilo_data = df[['Employee Full Name', 'date', 'Actual Login', 'Actual Logout', 'Employee ID']].copy()
lilo_data.dropna(subset=['Actual Login'], inplace=True)
lilo_data.dropna(subset=['Actual Logout'], inplace=True)
lilo_data.reset_index(drop=True, inplace=True)
lilo_data['Actual Login'] = lilo_data['Actual Login'].dt.strftime('%H:%M')
lilo_data['Actual Logout'] = lilo_data['Actual Logout'].dt.strftime('%H:%M')
lilo_data.head(10)

,Employee Full Name,date,Actual Login,Actual Logout,Employee ID
0,"Bennett, Samuel",12/06/2023,15:58,01:01,1544048
1,"Brooks, Wyatt",12/06/2023,22:59,08:00,1640652
2,"Caldwell, Ethan",12/06/2023,21:30,06:32,1785955
3,"Chandler, Amelia",12/06/2023,19:58,05:01,1897692
4,"Chandler, Ella",12/06/2023,21:29,06:41,1045515
5,"Cooper, Addison",12/06/2023,21:30,06:30,1116810
6,"Dawson, Aria",12/06/2023,05:58,15:00,1336897
7,"Donovan, Grayson",12/06/2023,05:59,15:01,1456507
8,"Donovan, Noah",12/06/2023,21:29,06:30,1705693
9,"Fitzgerald, Oliver",12/06/2023,21:58,07:01,1807780


This table is saved to a csv file.

In [22]:
lilo_data.to_csv('/content/drive/MyDrive/attendance/lilo_data.csv', index=False)

This data is now in a format that can be used in our attendance dashboard.

![picture](https://drive.google.com/uc?export=view&id=1onbM-jIHQ5Tp4O0xU9AzyzHDk-lpAd1X)